## argparser包

用途：编写命令行接口，对于程序所需参数，可以进行正确解析

注意：可以自动生成``help``和`usage`信息，当参数无效时，自动生成错误信息

官方文档：https://docs.python.org/3/library/argparse.html

### 1 主要步骤
```
step 1: 导入包   import argparse    
step 2: 创建对象 parser = argparse.ArgumentParser()    
step 3: 添加参数 parser.add_argument()    
step 4: 解析参数 args = parser.parse_args()
```

### 2 ArgumentParser对象

```
class argparse.ArgumentParser(prog=None,
                            usage=None,
                            description=None,
                            epilog=None,
                            parents=[],
                            formatter_class=argparse.HelpFormatter,
                            argument_default=None, 
                            conflict_handler=’error’,
                            add_help=True,
                            allow_abbrev=True)
````

- prog: 程序文件名，默认sys.argv[0]
- usage: 程序使用说明
- description: 程序目的说明，默认none
- epilog: 程序说明后记，默认none
- parents: ArgumentParser对象的父对象的参数列表
- formatter_class: help信息的说明格式
- argument_default: 参数的全局默认值
- conflict_handler: 冲突处理
- add_help: 是否增加help选项,默认True
- allow_abbrev: 是否使用参数的缩写,默认True

#### 2.1 prog参数:修改ArgumentParser对象的默认文件名
不论程序的文件名是来自sys.argv[0]，不是来自参数prog，都可以通过%(prog)s来引用程序的文件名。

In [1]:
# 不加prog参数
%run ArgumentParser参数/non_prog.py --help

usage: non_prog.py [-h] [--example EXAMPLE]

optional arguments:
  -h, --help         show this help message and exit
  --example EXAMPLE  Example help


In [2]:
%run ArgumentParser参数/prog.py --help

usage: Add prog [-h] [--example EXAMPLE]

optional arguments:
  -h, --help         show this help message and exit
  --example EXAMPLE  Example of the Add prog program


#### 2.2 usage参数：修改程序的使用说明信息
默认情况下，ArgumentParser对象会根据它所包括的参数来自动的生成使用说明

In [2]:
%run ArgumentParser参数/non_usage.py  --help

usage: Non usage [-h] [--example EXAMPLE] [--example1 [EXAMPLE1]]
                 example2 [example2 ...]

positional arguments:
  example2              Example2 help

optional arguments:
  -h, --help            show this help message and exit
  --example EXAMPLE     Example help
  --example1 [EXAMPLE1]
                        Example1 help


In [4]:
%run ArgumentParser参数/usage.py  --help

usage: Add usage [options]

positional arguments:
  example2              Example2 help

optional arguments:
  -h, --help            show this help message and exit
  --example EXAMPLE     Example help
  --example1 [EXAMPLE1]
                        Example1 help


### 3 ArgumentParser.add_argument

1. ArgumentParser.add_argument(name or flags…[, action][, nargs][, const][, default][, type][, choices][, required][, help][, metavar][, dest])
2. 通过add-argument()方法来给ArgumentParser对象添加新的命令行参数，参数的类型和相应的处理方法由不同的参数决定

- name or flags: 参数名或者参数标识,如foo or -f, –foo.
  - 带-的为可选参数（optional parameter）
  - 不带-的为必选参数（positional parametrer）。
- action: 参数的处理方法
- nargs: 参数的数量
- const: 参数的常量值
- default: 参数的默认值
- type: 参数的数据类型
- choices: 参数的取值范围
- required: 参数是否可以忽略不写，仅对可选参数有效
- help: 参数的说明信息
- metavar: 参数在说明信息usage中的名称
- dest: 对象的属性名

#### 3.1 name or flags
可选参数默认以-开始，其它的为必选参数

In [6]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-f', '--foo')
parser.add_argument('bar')
parser.parse_args(['BAR'])

Namespace(bar='BAR', foo=None)

In [7]:
parser.parse_args(['BAR', '--foo', 'FOO'])

Namespace(bar='BAR', foo='FOO')

In [8]:
parser.parse_args(['--foo', 'FOO']) # 会报错

usage: PROG [-h] [-f FOO] bar
PROG: error: the following arguments are required: bar


SystemExit: 2

/home/weijunfei/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### 3.2 actions
1. ArgumentParser对象将命令行参数与动作联系在一起，这些动作可以对命令行参数进行适当的处理,最常用的动作就是在parse_args()返回的对象中添加一个属性。
2. 参数action说明了命令行参数应该如何被处理
##### 'store':存储参数的值,在parser_args()方法返回的Namespace对象中添加属性，并设定它的值。它是action的默认值。

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
parser.parse_args('--foo 1'.split())

Namespace(foo='1')

##### 'store_const: 将命令行参数的值设定为参数const指定值

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_const', const=42)
parser.parse_args(['--foo'])

Namespace(foo=42)

##### 'store_true' and 'store_false': 将命令行参数的值设定为True或者False。

In [14]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--bar', action='store_false')
parser.add_argument('--baz', action='store_false')
parser.parse_args('--foo --bar'.split())

Namespace(bar=False, baz=True, foo=True)

##### 'append': 将命令行参数的值设定为一个列表，它可以用来处理同一个命令行参数出现多次的情况

In [15]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='append')
parser.parse_args('--foo 1 --foo 2'.split())

Namespace(foo=['1', '2'])

##### 'append_const': 将命令行参数的值设定为一个列表，列表中的每个值都来自参数const的取值。

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.add_argument('--int', dest='types', action='append_const', const=int)
parser.parse_args('--str --int'.split())

Namespace(types=[<class 'str'>, <class 'int'>])

##### 'count': 命令行参数的出现次数

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument('--verbose', '-v', action='count')
parser.parse_args(['-vvv'])

Namespace(verbose=3)

##### 'help': 命令行参数的帮助信息。
##### 'version': 和参数version用来说明相应的版本信息

In [25]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--version', action='version', version='%(prog)s 2.0')
parser.parse_args(args = ['--version'])

PROG 2.0


SystemExit: 0

#### 3.3 nargs

1. 使用不多
2. 一般情况下，每个命令行参数只与一个动作相联系，参数nargs可以让多个参数与一个动作都联系，即一个动作可以处理多个参数：N (an integer). N个参数被组合成一个列表

In [26]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs=2)
parser.add_argument('bar', nargs=1)
parser.parse_args('c --foo a b'.split())

Namespace(bar=['c'], foo=['a', 'b'])

#### 3.4 const
命令行参数的常量值，通常与append_const和store_const相关

### 3.5 default
必选参数和可选的参数的默认值

In [2]:
%run Add_parser参数/default.py
args

Namespace(foo='2')

参数默认值的解析与参数的类型有关

In [3]:
%run Add_parser参数/default_type.py
args

Namespace(length=10, width=10.5)

#### 3.6 type
命令行参数的数据类型,可以是任意Python支持的数据类型

In [6]:
%run Add_parser参数/type.py
args

usage: type.py [-h] foo bar
type.py: error: the following arguments are required: foo, bar


SystemExit: 2

Namespace(length=10, width=10.5)

### 4 parser.parse_args()

将命令行参数字符串转换为相应对象并赋值给Namespace对象的相应属性，默认返回一个Namespace对象。

ArgumentParser.parse_args(args=None, namespace=None)
- args: 字符串列表，默认来自sys.argv
- namespace: 对象名，默认是一个空Namespace对象。

调用parse_args()的返回值是一个Namespace对象，它具有很多属性，每个属性都代表相应的命令行参数。

Namespace对象是一个非常简单的类，可以通过vars()将之转换成字典类型

In [22]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
args = parser.parse_args(['--foo', 'BAR'])

In [23]:
args

Namespace(foo='BAR')

In [24]:
vars(args)

{'foo': 'BAR'}